In [107]:
import pandas as pd
import re

In [108]:
tsla = pd.read_csv('TSLA(2).csv')
elon = pd.read_csv('elonmusk.csv')

In [109]:
def separate_date(x):
    timestamp = pd.Timestamp(x)
    date = timestamp.date()
    time = timestamp.time()
    
    return date


def separate_time(x):
    timestamp = pd.Timestamp(x)
    date = timestamp.date()
    time = timestamp.time()

    return time


In [110]:
tsla['Date'] = tsla['Date'].apply(lambda x: pd.Timestamp(x).date())
elon['Date'] = elon.apply(lambda x: separate_date(x['Timestamp']), axis=1)
elon['Time'] = elon.apply(lambda x: separate_time(x['Timestamp']), axis=1)
elon['weekday'] = elon['Date'].apply(lambda x: x.weekday())
elon['is_weekend'] = elon['weekday'].apply(lambda x: 1 if x in [5, 6] else 0)
elon['trading_day'] = 0
elon.loc[elon['Date'].isin(tsla['Date']), ['trading_day']] = 1


In [111]:
tsla['Open'].describe()

count    2976.000000
mean      144.787077
std       260.019507
min         3.228000
25%        20.340000
50%        46.893999
75%        68.468502
max      1234.410034
Name: Open, dtype: float64

In [112]:
tsla['Close'].describe()

count    2976.000000
mean      144.824658
std       259.981333
min         3.160000
25%        20.256000
50%        46.824999
75%        68.574499
max      1229.910034
Name: Close, dtype: float64

In [113]:
tsla['Delta'] = tsla['Close'] / tsla['Open'] - 1
tsla['min_max_ratio'] = tsla['High'] / tsla['Low'] - 1

In [114]:
tsla[tsla['min_max_ratio'] <0]

,Date,Open,High,Low,Close,Adj Close,Volume,Delta,min_max_ratio


In [115]:
tsla[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', ]] = tsla[[
    'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',]].pct_change(fill_method='ffill')
    

In [116]:
tsla['Delta'].describe()

count    2975.000000
mean             NaN
std              NaN
min             -inf
25%        -1.987464
50%        -1.020795
75%        -0.046906
max              inf
Name: Delta, dtype: float64

In [117]:
def getFivePriorTradingDays(date):
    priors = tsla[tsla['Date'] < date]
    return priors['Date'].tail(5).tolist()
    
    

In [118]:
def getValues(dates, col):
    res = []
    for date in dates:
        res.append(tsla[tsla['Date'] == date][col].values[0])
    
    return res

In [119]:
elon['five_prior_trading_days'] = elon['Date'].apply(lambda x: getFivePriorTradingDays(x))

In [120]:
elon['five_prior_close'] = elon.apply(lambda x: getValues(x['five_prior_trading_days'], 'Close'), axis=1)
elon['five_prior_open'] = elon.apply(lambda x: getValues(x['five_prior_trading_days'], 'Open'), axis=1)
elon['five_prior_delta'] = elon.apply(lambda x: getValues(x['five_prior_trading_days'], 'Delta'), axis=1)
elon['five_prior_ratio'] = elon.apply(lambda x: getValues(x['five_prior_trading_days'], 'min_max_ratio'), axis=1)



KeyboardInterrupt: 

In [ ]:
usefulElon = elon[['text','Comments','Likes','Retweets','Date', 'Time','weekday', 'is_weekend', 'five_prior_trading_days','five_prior_open','five_prior_close','five_prior_delta','five_prior_ratio', 'trading_day']].loc[1:]

In [ ]:
def clean_text(x):
    temp = x.lower()
    temp = re.sub(r"http\S+", "", temp)
    temp = re.sub(r"www.\S+", "", temp)
    temp = re.sub(r"\n", " ", temp)
    temp = temp.rsplit(' ', 3)[0]
    return temp
    
    
    

In [ ]:
usefulElon['clean_text'] = usefulElon['text'].apply(lambda x: clean_text(x))

In [ ]:
def get_trading_info(row):
    if row['trading_day'] == 1:
        temp = tsla[tsla['Date'] == row['Date']]
    else:
        next_trading_day = tsla[tsla['Date'] > row['Date']].iloc[0]['Date']
        temp = tsla[tsla['Date'] == next_trading_day]
    
    return pd.Series(temp.values[0][1:])

In [ ]:
labels = usefulElon.copy()
labels[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume','Delta','min_max_ratio']] = labels.apply(
    lambda x: get_trading_info(x), axis=1)


In [ ]:
labels = labels[['Date','Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume','Delta','min_max_ratio']]

In [ ]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12205 entries, 1 to 12205
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          12205 non-null  object 
 1   Open          12205 non-null  float64
 2   High          12205 non-null  float64
 3   Low           12205 non-null  float64
 4   Close         12205 non-null  float64
 5   Adj Close     12205 non-null  float64
 6   Volume        12205 non-null  float64
 7   Delta         12205 non-null  float64
 8   min_max_diff  12205 non-null  float64
dtypes: float64(8), object(1)
memory usage: 858.3+ KB


In [ ]:
usefulElon.fillna(0, inplace=True)

In [ ]:
usefulElon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12205 entries, 1 to 12205
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   text                     12205 non-null  object
 1   Comments                 12205 non-null  object
 2   Likes                    12205 non-null  object
 3   Retweets                 12205 non-null  object
 4   Date                     12205 non-null  object
 5   Time                     12205 non-null  object
 6   weekday                  12205 non-null  int64 
 7   is_weekend               12205 non-null  int64 
 8   five_prior_trading_days  12205 non-null  object
 9   five_prior_open          12205 non-null  object
 10  five_prior_close         12205 non-null  object
 11  five_prior_delta         12205 non-null  object
 12  five_prior_diff          12205 non-null  object
 13  trading_day              12205 non-null  int64 
 14  clean_text               12205 non-nul

In [ ]:
labels.to_csv('lables.csv')
usefulElon.to_csv('features.csv')